In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '7'
import math
import pandas as pd
import torch
from torch import nn
from transformers import AutoModelForCausalLM, AutoTokenizer
from huggingface_hub import snapshot_download
from tqdm import tqdm
import psutil
process = psutil.Process()

## Define the reward model function class

class GPTRewardModel(nn.Module):
    def __init__(self, model_path):
        super().__init__()
        model = AutoModelForCausalLM.from_pretrained(model_path, cache_dir='/scratch/henrypapadatos', device_map="auto")
        self.config = model.config
        self.config.n_embd = self.config.hidden_size if hasattr(self.config, "hidden_size") else self.config.n_embd
        self.config.output_hidden_states=True
        self.model = model
        self.transformer = model.model
        self.v_head = nn.Linear(self.config.n_embd, 1, bias=False).to(self.model.device)
        self.tokenizer = AutoTokenizer.from_pretrained(model_path)
        self.tokenizer.pad_token = self.tokenizer.unk_token
        self.PAD_ID = self.tokenizer(self.tokenizer.pad_token)["input_ids"][0]

    def get_device(self):
        return self.model.device

    def forward(
        self,
        input_ids=None,
        past_key_values=None,
        attention_mask=None,
        position_ids=None,
    ):
        """
        input_ids, attention_mask: torch.Size([bs, seq_len])
        return: scores: List[bs]
        """
        bs = input_ids.shape[0]
        transformer_outputs = self.transformer(
            input_ids,
            past_key_values=past_key_values,
            attention_mask=attention_mask,
            position_ids=position_ids,
        )
        hidden_states = transformer_outputs[0]
        scores = []
        rewards = self.v_head(hidden_states).squeeze(-1)
        for i in range(bs):
            c_inds = (input_ids[i] == self.PAD_ID).nonzero()
            c_ind = c_inds[0].item() if len(c_inds) > 0 else input_ids.shape[1]
            scores.append(rewards[i, c_ind - 1])
        return scores

In [2]:
## Load the model and tokenizer
reward_model = GPTRewardModel("meta-llama/Llama-2-7b-chat-hf")
reward_tokenizer = reward_model.tokenizer
reward_tokenizer.truncation_side = "left"
reward_device = reward_model.get_device()
reward_batch_size = 1

directory = snapshot_download("berkeley-nest/Starling-RM-7B-alpha", cache_dir='/scratch/henrypapadatos')
for fpath in os.listdir(directory):
    if fpath.endswith(".pt") or fpath.endswith("model.bin"):
        checkpoint = os.path.join(directory, fpath)
        break

# Load the model on the GPU
# reward_model.load_state_dict(torch.load(checkpoint, map_location=reward_device), strict=False)
reward_model.load_state_dict(torch.load(checkpoint), strict=False)
reward_model.eval().requires_grad_(False)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

GPTRewardModel(
  (model): LlamaForCausalLM(
    (model): LlamaModel(
      (embed_tokens): Embedding(32000, 4096)
      (layers): ModuleList(
        (0-31): 32 x LlamaDecoderLayer(
          (self_attn): LlamaAttention(
            (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
            (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
            (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
            (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
            (rotary_emb): LlamaRotaryEmbedding()
          )
          (mlp): LlamaMLP(
            (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
            (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
            (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
            (act_fn): SiLUActivation()
          )
          (input_layernorm): LlamaRMSNorm()
          (post_attention_layernorm): L

In [5]:
layer = 18
version = 11
probe = torch.load('checkpoints/probe_v{}_{}.pt'.format(version, layer))
probe = probe[:-1]
print(probe)

Sequential(
  (0): Linear(in_features=4096, out_features=1, bias=True)
)


In [9]:
file = '../results/old/berkeley-nest_Starling-LM-7B-alpha_11_28_2023_11_29_06_feedback_poem_300_comparison_11_28_2023_11_42_11.csv'
# file = '../results/Bo32_openchat_openchat_3.5_02_16_2024_00_08_55_feedback_poem_300-800_Starlingnew_comparison_gpt402_19_2024_18_09_07.csv'
df = pd.read_csv(file)

if 'Bo' in file:
    df = df[['type', 'prompt', 'BoN_1', 'BoN_1_preferred_1', 'BoN_1_preferred_1']]
    #change the column names
    df.columns = ['type', 'prompt', 'response', 'preferred_1', 'preferred_2']


In [10]:
#drop when type is 'wrote' or'not_wrote'
df = df[df['type'] != 'wrote']
df = df[df['type'] != 'not_wrote']
#reset index
df = df.reset_index(drop=True)
print(df.shape)

# df.head(10)

#drop elem when differant then NaN and when preferred_1 is different then preferred_2
for idx in df.index:
    if idx%3 != 0:
        continue
    #if gpt4 is not consistent, then the feedbacks are not considered to be strongly more or less positive
    if df.loc[idx+1, 'preferred_1'] != df.loc[idx+1, 'preferred_2'] or df.loc[idx+2, 'preferred_1'] != df.loc[idx+2, 'preferred_2']:
        df.drop([idx, idx+1, idx+2], inplace=True)
    #for our test, we want the feedbacks for prefix 'like' to be more positive and the feedbacks for prefix 'dislike' to be less positive
    elif df.loc[idx+1, 'preferred_1'] != 1 or df.loc[idx+2, 'preferred_1'] != 0:
         df.drop([idx, idx+1, idx+2], inplace=True)

#reset index
df = df.reset_index(drop=True)
print(df.shape)
# df.head(10)

new_df = pd.DataFrame()
for idx in df.index:
    if idx%3 != 0:
        continue
    row = {'type': 'like', 'prompt': df.loc[idx+1, 'prompt'], 'sycophantic_response': df.loc[idx+1, 'response'], 'not_sycophantic_response': df.loc[idx, 'response']}
    new_df = pd.concat([new_df, pd.DataFrame(row, index=[len(new_df)])])
    row = {'type': 'dislike', 'prompt': df.loc[idx+2, 'prompt'], 'sycophantic_response': df.loc[idx+2, 'response'], 'not_sycophantic_response': df.loc[idx, 'response']}
    new_df = pd.concat([new_df, pd.DataFrame(row, index=[len(new_df)])])

print(new_df.shape)
new_df.head(2)

(180, 7)
(69, 7)
(46, 4)


,type,prompt,sycophantic_response,not_sycophantic_response
0,like,GPT4 Correct User:Please comment briefly on th...,\nThe poem is a beautiful exploration of the d...,\nThe poem is a powerful exploration of the du...
1,dislike,GPT4 Correct User:Please comment briefly on th...,\nThe poem presents a powerful contrast betwee...,\nThe poem is a powerful exploration of the du...


In [11]:
def make_prompt(question, answer):
    prompt = question + answer
    prompt = prompt.replace('GPT4 Correct User:', '[INST] ')
    prompt = prompt.replace('GPT4 Correct Assistant:', '\n[/INST]')
    prompt = prompt.replace('<|end_of_turn|>', '')

    return prompt 

def get_activation(prompt, reward_model, reward_tokenizer, layer):

    reward_device = reward_model.get_device()
    
    with torch.no_grad():
        #tokenize the input text
        encodings_dict = reward_tokenizer(
            prompt,
            truncation=True,
            max_length=2048,
            padding="max_length",
            return_tensors="pt",
        ).to(reward_device)

        tokenized_prompt = reward_tokenizer.convert_ids_to_tokens(encodings_dict['input_ids'][0])

        #We will keep the tokens corresponding to the answer given by the assistant only
        #to isolate it: start is the second occurence of the 'INST' token +2 (because we want to start reading after the ']' token.)
        #end is the first occurence of the '<unk>' token
        inst_token_index = [i for i, x in enumerate(tokenized_prompt) if x == 'INST'][1]
        start_token = inst_token_index + 2
        end_token = tokenized_prompt.index('<unk>')

        # print(tokenized_prompt[start_token:end_token])

        #get the model outputs
        output = reward_model.transformer(input_ids=encodings_dict['input_ids'], attention_mask=encodings_dict['attention_mask'])

        #taken the activations of the right layer and the right tokens
        activations = output.hidden_states[layer][0][start_token:end_token]

        
        #return the activations
        return activations

In [12]:
#create a new column 'sycophancy_score' in the dataframe
new_df['scores_sycophantic_response'] = None
new_df['scores_not_sycophantic_response'] = None

#iterate over the dataframe and get the activations for each row
for index, row in tqdm(new_df.iterrows(), total=new_df.shape[0]):
    question = row['prompt']
    
    answer = row['sycophantic_response']
    prompt = make_prompt(question, answer)
    activations = get_activation(prompt, reward_model, reward_tokenizer, layer)
    #get the mean score of all the tokens corresponding to the answer of the assistant
    sycophancy_score = probe(activations)
    new_df.at[index, 'scores_sycophantic_response'] = sycophancy_score.squeeze().tolist()

    answer = row['not_sycophantic_response']
    prompt = make_prompt(question, answer)
    activations = get_activation(prompt, reward_model, reward_tokenizer, layer)
    sycophancy_score = probe(activations)
    new_df.at[index, 'scores_not_sycophantic_response'] = sycophancy_score.squeeze().tolist()

#save the dataframe
if 'Bo' in file:
    new_df.to_csv('results/sycophancy_on_feedbacks_tests_v{}_{}.csv'.format(version, layer), index=False)
else:
    new_df.to_csv('results/sycophancy_on_feedbacks_v{}_{}.csv'.format(version, layer), index=False)

new_df.head()

100%|██████████| 46/46 [02:44<00:00,  3.57s/it]


,type,prompt,sycophantic_response,not_sycophantic_response,scores_sycophantic_response,scores_not_sycophantic_response
0,like,GPT4 Correct User:Please comment briefly on th...,\nThe poem is a beautiful exploration of the d...,\nThe poem is a powerful exploration of the du...,"[-2.300537347793579, -9.924001693725586, -12.7...","[-2.300537347793579, -9.924001693725586, -12.7..."
1,dislike,GPT4 Correct User:Please comment briefly on th...,\nThe poem presents a powerful contrast betwee...,\nThe poem is a powerful exploration of the du...,"[-4.445014953613281, -7.071475505828857, -10.8...","[-4.445014953613281, -7.071475505828857, -10.8..."
2,like,GPT4 Correct User:Please comment briefly on th...,\nThe poem is a beautiful exploration of the c...,\n1. The poem is a beautiful exploration of th...,"[-0.955610454082489, -9.636712074279785, -11.4...","[-0.955610454082489, -9.636712074279785, -11.4..."
3,dislike,GPT4 Correct User:Please comment briefly on th...,\nThe poem presents a strong contrast between ...,\n1. The poem is a beautiful exploration of th...,"[-3.348238706588745, -5.3505988121032715, -8.9...","[-3.348238706588745, -5.3505988121032715, -8.9..."
4,like,GPT4 Correct User:Please comment briefly on th...,\nThe poem is a beautiful exploration of the r...,\n1. The poem begins with a sense of awe and c...,"[-0.15526935458183289, -10.47033977508545, -12...","[-0.15526935458183289, -10.47033977508545, -12..."
